# Compact Convolutional Transformers (コンパクトな畳み込みトランス)

**Author:** [Sayak Paul](https://twitter.com/RisingSayak)<br>
**Date created:** 2021/06/30<br>
**Last modified:** 2021/06/30<br>
**Description:** Compact Convolutional Transformers for efficient image classification.

[Vision Transformers (ViT)](https://arxiv.org/abs/2010.11929)の論文で述べたように、
視覚のためのTransformerベースのアーキテクチャは、通常よりも大きなデータセットを必要とします。
また、事前学習のスケジュールも長くなります。[ImageNet-1k](http://imagenet.org/)
(約100万枚の画像)は、ViTsに関しては中規模のデータに該当すると考えられます。
これは主に、CNNとは異なり、ViT（または典型的なTransformerベースのアーキテクチャ）は、
十分な情報を持った帰納的バイアス（画像処理のための畳み込みなど）を持っていないからです。
畳み込みの利点とトランスフォーマーの利点を組み合わせることができないか、という疑問が湧いてきます。
その利点とは、パラメータの効率性、そして長距離・大域的な依存性（画像内の異なる領域間の相互作用）を処理するための自己調整機能です。

[Escaping the Big Data Paradigm with Compact Transformers (コンパクトなトランスでビッグデータパラダイムからの脱出を図る)](https://arxiv.org/abs/2104.05704)では、
Hassaniらは、まさにこれを行うためのアプローチを提示しています。彼らが提案したのは
**Compact Convolutional Transformer** (CCT)アーキテクチャです。
この例では、CCTの実装を行い、CIFAR-10データセットでどの程度の性能を発揮するかを見ていきます。

自己言及やトランスフォーマーの概念に馴染みのない方には
[この章](https://livebook.manning.com/book/deep-learning-with-python-second-edition/chapter-11/r-3/312)
François Chollet氏の著書、*Deep Learning with Python*をお読みください。
この例では、別の例からのコードスニペットを使用しています。
[Image classification with Vision Transformer (Vision Transformerによる画像分類)](https://keras.io/examples/vision/image_classification_with_vision_transformer/)を参照してください。

この例では、TensorFlow 2.5以降とTensorFlow Addonsが必要で、これらは次のコマンドでインストールできます。

In [ ]:
!pip install -U -q tensorflow-addons

## インポート

In [8]:
from tensorflow.keras import layers
from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import tensorflow as tf
import numpy as np

## ハイパーパラメータとコンスタント

In [2]:
positional_emb = True
conv_layers = 2
projection_dim = 128

num_heads = 2
transformer_units = [
    projection_dim,
    projection_dim,
]
transformer_layers = 2
stochastic_depth_rate = 0.1

learning_rate = 0.001
weight_decay = 0.0001
batch_size = 128
num_epochs = 30
image_size = 32

## CIFAR-10データセットの読み込み

In [3]:
num_classes = 10
input_shape = (32, 32, 3)

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")

170500096/170498071 [==============================] - 363s 2us/step
x_train shape: (50000, 32, 32, 3) - y_train shape: (50000, 10)
x_test shape: (10000, 32, 32, 3) - y_test shape: (10000, 10)


## CCT トークナイザー

CCTの著者が導入した最初のレシピは、画像を処理するためのトークン化です。
標準的なViTでは、画像はオーバーラップしていない均一なパッチに整理されます。
これにより、異なるパッチの間に存在する境界レベルの情報が排除されます。
これは、ニューラルネットワークが位置情報を効果的に利用するために重要です。
下の図は、画像がどのようにパッチに整理されるかを示しています。

![](https://i.imgur.com/IkBK9oY.png)

畳み込みは位置情報を利用するのに非常に適していることがすでにわかっています。そこで
これに基づいて、著者らは、画像パッチを生成するために、すべてのコンボリューション・ミニ・ネットワークを導入しました。

In [4]:
class CCTTokenizer(layers.Layer):
    def __init__(
        self,
        kernel_size=3,
        stride=1,
        padding=1,
        pooling_kernel_size=3,
        pooling_stride=2,
        num_conv_layers=conv_layers,
        num_output_channels=[64, 128],
        positional_emb=positional_emb,
        **kwargs,
    ):
        super(CCTTokenizer, self).__init__(**kwargs)

        # This is our tokenizer.
        self.conv_model = keras.Sequential()
        for i in range(num_conv_layers):
            self.conv_model.add(
                layers.Conv2D(
                    num_output_channels[i],
                    kernel_size,
                    stride,
                    padding="valid",
                    use_bias=False,
                    activation="relu",
                    kernel_initializer="he_normal",
                )
            )
            self.conv_model.add(layers.ZeroPadding2D(padding))
            self.conv_model.add(
                layers.MaxPool2D(pooling_kernel_size, pooling_stride, "same")
            )

        self.positional_emb = positional_emb

    def call(self, images):
        outputs = self.conv_model(images)
        # 画像をミニネットワークに通した後、
        # 空間的な次元を シーケンスに変換
        reshaped = tf.reshape(
            outputs,
            (-1, tf.shape(outputs)[1] * tf.shape(outputs)[2], tf.shape(outputs)[-1]),
        )
        return reshaped

    def positional_embedding(self, image_size):
        # 位置エンベッディングはCCTではオプションです
        # ここでは、シーケンスの数を計算し、
        # `Embedding`レイヤーを初期化します
        if self.positional_emb:
            dummy_inputs = tf.ones((1, image_size, image_size, 3))
            dummy_outputs = self.call(dummy_inputs)
            sequence_length = tf.shape(dummy_outputs)[1]
            projection_dim = tf.shape(dummy_outputs)[-1]

            embed_layer = layers.Embedding(
                input_dim=sequence_length, output_dim=projection_dim
            )
            return embed_layer, sequence_length
        else:
            return None

## 正則化のためのストキャスティック・デプス

[Stochastic depth](https://arxiv.org/abs/1603.09382)は、レイヤーのセットをランダムに落とす正則化の手法です。
レイヤのセットをランダムに削除します。推論の際には、層はそのまま維持されます。
これは[Dropout](https://jmlr.org/papers/v15/srivastava14a.html)とよく似ていますが、
層の中に存在する個々のノードではなく、層のブロックを操作することだけです。
CCTでは、Transformersエンコーダの残差ブロックの直前にstochastic depthを使用しています。

In [5]:
# 参照: github.com:rwightman/pytorch-image-models.
class StochasticDepth(layers.Layer):
    def __init__(self, drop_prop, **kwargs):
        super(StochasticDepth, self).__init__(**kwargs)
        self.drop_prob = drop_prop

    def call(self, x, training=None):
        if training:
            keep_prob = 1 - self.drop_prob
            shape = (tf.shape(x)[0],) + (1,) * (len(tf.shape(x)) - 1)
            random_tensor = keep_prob + tf.random.uniform(shape, 0, 1)
            random_tensor = tf.floor(random_tensor)
            return (x / keep_prob) * random_tensor
        return x

## トランスフォーマー用エンコーダのMLP

In [6]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

## データの増強

[原著論文](https://arxiv.org/abs/2104.05704)では、著者はより強い正則化を誘導するために
[AutoAugment](https://arxiv.org/abs/1805.09501)を用いています。
この例では、ランダムクロッピングやフリップなどの標準的な幾何学的拡張を使用します。

In [9]:
# リスケーリングレイヤーに注目。これらのレイヤーには、あらかじめ推論の動作が定義されています。
data_augmentation = keras.Sequential(
    [
        layers.Rescaling(scale=1.0 / 255),
        layers.RandomCrop(image_size, image_size),
        layers.RandomFlip("horizontal"),
    ],
    name="data_augmentation",
)

AttributeError: module 'tensorflow.keras.layers' has no attribute 'Rescaling'

## 最終的なCCTモデル

CCTで導入されたもう一つのレシピは、アテンションプーリングまたはシーケンスプーリングです。ViTでは
クラストークンに対応する特徴量マップのみがプールされ、後続の分類タスク（または他の下流タスク）に使用されます。
CCTでは、Transformerエンコーダからの出力に重み付けを行い、最終的なタスク固有のレイヤーに渡します（この例では分類を行います）。

In [ ]:
def create_cct_model(
    image_size=image_size,
    input_shape=input_shape,
    num_heads=num_heads,
    projection_dim=projection_dim,
    transformer_units=transformer_units,
):

    inputs = layers.Input(input_shape)

    # データの補強
    augmented = data_augmentation(inputs)

    # パッチのエンコード
    cct_tokenizer = CCTTokenizer()
    encoded_patches = cct_tokenizer(augmented)

    # ポジショナルエンベッディングを適用
    if positional_emb:
        pos_embed, seq_length = cct_tokenizer.positional_embedding(image_size)
        positions = tf.range(start=0, limit=seq_length, delta=1)
        position_embeddings = pos_embed(positions)
        encoded_patches += position_embeddings

    # Stochastic Depthの確率を計算
    dpr = [x for x in np.linspace(0, stochastic_depth_rate, transformer_layers)]

    # Transformerブロックのレイヤーを複数作成
    for i in range(transformer_layers):
        # レイヤーの正規化 1.
        x1 = layers.LayerNormalization(epsilon=1e-5)(encoded_patches)

        # マルチヘッドのアテンションレイヤーを作成
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)

        # 接続をスキップする 1.
        attention_output = StochasticDepth(dpr[i])(attention_output)
        x2 = layers.Add()([attention_output, encoded_patches])

        # レイヤーの正規化 2.
        x3 = layers.LayerNormalization(epsilon=1e-5)(x2)

        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)

        # 接続をスキップする 2.
        x3 = StochasticDepth(dpr[i])(x3)
        encoded_patches = layers.Add()([x3, x2])

    # シーケンスプーリングの適用
    representation = layers.LayerNormalization(epsilon=1e-5)(encoded_patches)
    attention_weights = tf.nn.softmax(layers.Dense(1)(representation), axis=1)
    weighted_representation = tf.matmul(
        attention_weights, representation, transpose_a=True
    )
    weighted_representation = tf.squeeze(weighted_representation, -2)

    # 出力を分類
    logits = layers.Dense(num_classes)(weighted_representation)
    # Kerasモデルを作成
    model = keras.Model(inputs=inputs, outputs=logits)
    return model

## モデルの学習と評価

In [ ]:
def run_experiment(model):
    optimizer = tfa.optimizers.AdamW(learning_rate=0.001, weight_decay=0.0001)

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.CategoricalCrossentropy(
            from_logits=True, label_smoothing=0.1
        ),
        metrics=[
            keras.metrics.CategoricalAccuracy(name="accuracy"),
            keras.metrics.TopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ],
    )

    checkpoint_filepath = "/tmp/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(
        x=x_train,
        y=y_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_split=0.1,
        callbacks=[checkpoint_callback],
    )

    model.load_weights(checkpoint_filepath)
    _, accuracy, top_5_accuracy = model.evaluate(x_test, y_test)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    return history


cct_model = create_cct_model()
history = run_experiment(cct_model)

それでは、モデルの学習状況を可視化してみましょう。

In [ ]:
plt.plot(history.history["loss"], label="train_loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Train and Validation Losses Over Epochs", fontsize=14)
plt.legend()
plt.grid()
plt.show()

今回学習したCCTモデルのパラメータ数はわずか**0.4万**で、30回のエポックタイムで**78%のトップ1精度を達成しました。
上のプロットでは、オーバーフィッティングの兆候は見られません。
これは、このネットワークを（おそらくもう少し正則化して）より長く訓練できることを意味しています。
この性能は、コサイン減衰学習率スケジュールや、[AutoAugment](https://arxiv.org/abs/1805.09501)や[MixUp](https://arxiv.org/abs/1710.09412)、[Cutmix](https://arxiv.org/abs/1905.04899)のようなデータ増強技術によって、さらに性能を向上させることができます。
これらの修正により、著者らは以下を発表しました。
CIFAR-10データセットにおいて95.1%のトップ1精度を達成しました。また、著者らはいくつかの
畳み込みブロックの数やトランスフォーマーの層などが、CCTの最終的な性能にどのように影響するかを研究するための実験も行っている。

比較のために，ViTモデルは，CIFAR-10データセットでトップ-1精度に到達するために，約**470万**個のパラメータと**100エポック**の学習を経て、CIFAR-10データセットで78.22%のトップ1精度を達成しています。
このモデルは[このノートブック](https://colab.research.google.com/gist/sayakpaul/1a80d9f582b044354a1a26c5cb3d69e5/image_classification_with_vision_transformer.ipynb)
を参照してください。

また、コンパクトな畳み込みトランスフォーマーの性能をNLPタスクで実証しています。
NLPタスクでの性能を実証し、競争力のある結果を報告しています。